In [ ]:
from utils import read_images, return_stacks_per_position_id, make_isotropic
import napari
import pyclesperanto_prototype as cle
from skimage.measure import regionprops_table
import pandas as pd
import napari_segment_blobs_and_things_with_membranes as nsbatwm

In [ ]:
# Define the path containing your stacks, the position id you want to explore and the scaling in x, y and z.
data_path = "./raw_data/ANP32A_N4/Crop"
position_id = 1
scaling_x_um = 0.342
scaling_y_um = 0.342
scaling_z_um = 0.663

# Define the analysis parameters
top_hat_radius = 5
gaussian_sigma = 2
voronoi_otsu_spot_sigma = 10
voronoi_otsu_outline_sigma = 1
closing_labels_radius = 5
erosion_labels_radius = 3

In [ ]:
# Retrieve the paths of all the images within data_path (ignoring ch02, brightfield channel)
images_per_position = read_images(data_path)

# Generate the ch00_stack (nuclei) and ch01_stack (protein of interest) from the individual slices 
ch00_stack, ch01_stack = return_stacks_per_position_id(images_per_position, position_id)

In [ ]:
# Rescale the ch00_stack (nuclei) and ch01_stack to make data isotropic in order to apply voronoi_otsu labeling afterwards
nuclei_resampled = make_isotropic(ch00_stack, scaling_x_um, scaling_y_um, scaling_z_um)
marker_resampled = make_isotropic(ch01_stack, scaling_x_um, scaling_y_um, scaling_z_um)

# Remove background with a top_hat_filter
background_subtracted = cle.top_hat_box(nuclei_resampled, radius_x=5, radius_y=5, radius_z=5)

# Apply gaussian blur to prevent the formation of holes upon labeling
post_gaussian = cle.gaussian_blur(background_subtracted, sigma_x=2, sigma_y=2, sigma_z=2)

# Voronoi-Otsu labeling
segmented = cle.voronoi_otsu_labeling(post_gaussian, spot_sigma=10, outline_sigma=1)

# Close holes in labels to avoid false emtpy volumes within the nuclei
closed_labels = cle.closing_labels(segmented, radius=5)

# Erode the labels to avoid quantifying protein of interest spots in close proximity but not inside the nucleus
eroded_labels = cle.erode_labels(closed_labels, radius=3)

# In order to remove labels on edges I have to remove the labels from the first and last z-planes
# Otherwise upon applying remove_labels_on_edges (skimage.clear_border) the nuclei touching "up" and "down" will be removed

# Pull eroded_labels from GPU to manipulate it as a np.array
eroded_labels_np = cle.pull(eroded_labels)

# Set the first and last slices to 0
eroded_labels_np[0, :, :] = 0
eroded_labels_np[-1, :, :] = 0

# Once the top and bottom slices are empty (no labels) we can apply remove on edges
remove_border_labels = nsbatwm.remove_labels_on_edges(eroded_labels_np)

In [ ]:
remove_border_labels_np = cle.pull(remove_border_labels)
marker_resampled_np = cle.pull(marker_resampled)

# Extract regionprops from labels not touching the borders
props = regionprops_table(label_image=remove_border_labels_np, intensity_image=marker_resampled_np, properties=["label", "intensity_mean", "intensity_max", "centroid", "area_filled"])

# Construct a dataframe
df = pd.DataFrame(props)

df

In [ ]:
intensity_mean = df["intensity_mean"].mean()
intensity_mean

In [ ]:
# Initialize napari.Viewer and display input stacks and label processing steps
viewer = napari.Viewer(ndisplay=2)


viewer.add_image(nuclei_resampled)
viewer.add_image(marker_resampled)
viewer.add_labels(segmented)
viewer.add_labels(closed_labels)
viewer.add_labels(eroded_labels)
viewer.add_labels(remove_border_labels)